# tea5poon

In [1]:
import os
import numpy as np
import tensorflow as wtf
import tensorflow.contrib.rnn as rnn
import data_utils as du

/home/noodles/.conda/envs/turntable/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
EPOCHS = 32

num_train, num_val, num_test = 70, 10, 20
train_data,val_data,test_data = du.create_dicts(num_train, num_val, num_test)

In [3]:
du.what_tf_g7_adaproon('val', val_data)
du.what_tf_g7_adaproon('test', test_data)
du.what_tf_g7_adaproon('train', train_data)

patient 0
example 17
patient 1
example 37
patient 2
example 57
patient 3
example 75
patient 4
example 95
patient 5
example 115
patient 6
example 135
patient 7
example 153
patient 8
example 165
patient 9
example 187
patient 0
example 20
patient 1
example 38
patient 2
example 58
patient 3
example 80
patient 4
example 92
patient 5
example 114
patient 6
example 126
patient 7
example 138
patient 8
example 158
patient 9
example 178
patient 10
example 195
patient 11
example 212
patient 12
example 232
patient 13
example 252
patient 14
example 270
patient 15
example 290
patient 16
example 304
patient 17
example 318
patient 18
example 338
patient 19
example 358
patient 0
example 20
patient 1
example 40
patient 2
example 60
patient 3
example 81
patient 4
example 99
patient 5
example 119
patient 6
example 139
patient 7
example 159
patient 8
example 179
patient 9
example 199
patient 10
example 219
patient 11
example 238
patient 12
example 259
patient 13
example 278
patient 14
example 298
patient 15

In [ ]:
MODEL_TYPE = 'tea5poon'
OVERFIT = False

MODEL_NAME = MODEL_TYPE + '-000'

BATCH_SIZE = 1

In [ ]:
def encoder_fn(inputs):
    conv1_1 = wtf.layers.conv2d(inputs,
                                filters=64,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv1_1')
    conv1_2 = wtf.layers.conv2d(conv1_1,
                                filters=64,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv1_2')
    pool1 = wtf.layers.max_pooling2d(conv1_2,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool1')
        
    conv2_1 = wtf.layers.conv2d(pool1,
                                filters=128,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv2_1')
    conv2_2 = wtf.layers.conv2d(conv2_1,
                                filters=128,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv2_2')
    pool2 = wtf.layers.max_pooling2d(conv2_2,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool2')
        
    conv3_1 = wtf.layers.conv2d(pool2,
                                filters=256,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv3_1')
    conv3_2 = wtf.layers.conv2d(conv3_1,
                                filters=256,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv3_2')
    conv3_3 = wtf.layers.conv2d(conv3_2,
                                filters=256,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv3_3')
    pool3 = wtf.layers.max_pooling2d(conv3_3,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool3')
        
    conv4_1 = wtf.layers.conv2d(pool3,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv4_1')
    conv4_2 = wtf.layers.conv2d(conv4_1,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv4_2')
    conv4_3 = wtf.layers.conv2d(conv4_2,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv4_3')
    pool4 = wtf.layers.max_pooling2d(conv4_3,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool4')
    
    conv5_1 = wtf.layers.conv2d(pool4,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv5_1')
    conv5_2 = wtf.layers.conv2d(conv5_1,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv5_2')
    conv5_3 = wtf.layers.conv2d(conv5_2,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv5_3')
    pool5 = wtf.layers.max_pooling2d(conv5_3,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool5')
    
    conv6_1 = wtf.layers.conv2d(pool5,
                                filters=1,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv6_1')
        
    dout = wtf.layers.dropout(conv6_1,
                              rate=0.0)
    
    outputs = dout
    return outputs, pool1, pool3, pool4

In [ ]:
def decoder_fn(inputs, pool1, pool3, pool4):
    deconv1 = wtf.layers.conv2d_transpose(inputs,
                                          filters=pool4.get_shape().as_list()[-1],
                                          kernel_size=4,
                                          strides=2,
                                          padding='same')
    deconv1_skip = wtf.add(deconv1, pool4)
    
    deconv2 = wtf.layers.conv2d_transpose(deconv1_skip,
                                          filters=pool3.get_shape().as_list()[-1],
                                          kernel_size=4,
                                          strides=2,
                                          padding='same')
    deconv2_skip = wtf.add(deconv2, pool3)
    
    deconv3 = wtf.layers.conv2d_transpose(deconv2_skip,
                                          filters=1,
                                          kernel_size=16,
                                          strides=8,
                                          padding='same')
#     deconv3_skip = wtf.add(deconv3, pool1)
    
    outputs = deconv3
    return outputs

In [ ]:
# t0,t1,t2 -> t3 + LSTM

def model_fn(features,
             labels,
             mode,
             params):
    with wtf.device('/device:GPU:1'):
        inputs = features
        print(inputs)
        inputs_tr = wtf.transpose(inputs, [2, 3, 4, 1, 0])
        inputs_re = wtf.reshape(inputs_tr, [192, 224, 3, -1])
        inputs_NHWC = wtf.transpose(inputs_re, [3, 0, 1, 2])
        print(inputs_NHWC)
        dout, pool1, pool3, pool4 = encoder_fn(inputs_NHWC) # dout (15*N, 6, 7, 1)
        dout_tr = wtf.transpose(dout, [1, 2, 3, 0])
        dout_re = wtf.reshape(dout_tr, [6*7*1, 15, -1])
        dout_NTC = wtf.transpose(dout_re, [2, 1, 0])
        lstm_cell = rnn.LSTMCell(num_units=6*7*1,
                                 forget_bias=1.0,
                                 reuse=None,
                                 name=None)
        rnn_outputs, _ = wtf.nn.dynamic_rnn(cell=lstm_cell,
                                            inputs=dout_NTC,
                                            dtype=wtf.float32)
#         print(rnn_outputs) # I'm not sure what shape the rnn outputs
        rnn_tr = wtf.transpose(rnn_outputs, [2, 1, 0])
        rnn_re = wtf.reshape(rnn_tr, [6, 7, 1, -1])
        rnn_NHWC = wtf.transpose(rnn_re, [3, 0, 1, 2])
        print(rnn_NHWC)
        deconv3 = decoder_fn(rnn_NHWC, pool1, pool3, pool4)
        dc3_tr = wtf.transpose(deconv3, [1, 2, 3, 0])
        dc3_re = wtf.reshape(dc3_tr, [192, 224, 1, 15, -1])
        outputs = wtf.transpose(dc3_re, [4, 3, 0, 1, 2])
        print(outputs)
        
        # Adds concatenated inputs and outputs to be displayed in TensorBoard
        disp00 = wtf.concat([wtf.expand_dims(inputs[:, 0,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 0,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 0,:,:,2], -1),
                             labels[:, 0], outputs[:, 0]], 1)
        disp01 = wtf.concat([wtf.expand_dims(inputs[:, 1,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 1,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 1,:,:,2], -1),
                             labels[:, 1], outputs[:, 1]], 1)
        disp02 = wtf.concat([wtf.expand_dims(inputs[:, 2,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 2,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 2,:,:,2], -1),
                             labels[:, 2], outputs[:, 2]], 1)
        disp03 = wtf.concat([wtf.expand_dims(inputs[:, 3,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 3,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 3,:,:,2], -1),
                             labels[:, 3], outputs[:, 3]], 1)
        disp04 = wtf.concat([wtf.expand_dims(inputs[:, 4,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 4,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 4,:,:,2], -1),
                             labels[:, 4], outputs[:, 4]], 1)
        disp05 = wtf.concat([wtf.expand_dims(inputs[:, 5,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 5,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 5,:,:,2], -1),
                             labels[:, 5], outputs[:, 5]], 1)
        disp06 = wtf.concat([wtf.expand_dims(inputs[:, 6,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 6,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 6,:,:,2], -1),
                             labels[:, 6], outputs[:, 6]], 1)
        disp07 = wtf.concat([wtf.expand_dims(inputs[:, 7,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 7,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 7,:,:,2], -1),
                             labels[:, 7], outputs[:, 7]], 1)
        disp08 = wtf.concat([wtf.expand_dims(inputs[:, 8,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 8,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 8,:,:,2], -1),
                             labels[:, 8], outputs[:, 8]], 1)
        disp09 = wtf.concat([wtf.expand_dims(inputs[:, 9,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 9,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 9,:,:,2], -1),
                             labels[:, 9], outputs[:, 9]], 1)
        disp10 = wtf.concat([wtf.expand_dims(inputs[:,10,:,:,0], -1),
                             wtf.expand_dims(inputs[:,10,:,:,1], -1),
                             wtf.expand_dims(inputs[:,10,:,:,2], -1),
                             labels[:,10], outputs[:,10]], 1)
        disp11 = wtf.concat([wtf.expand_dims(inputs[:,11,:,:,0], -1),
                             wtf.expand_dims(inputs[:,11,:,:,1], -1),
                             wtf.expand_dims(inputs[:,11,:,:,2], -1),
                             labels[:,11], outputs[:,11]], 1)
        disp12 = wtf.concat([wtf.expand_dims(inputs[:,12,:,:,0], -1),
                             wtf.expand_dims(inputs[:,12,:,:,1], -1),
                             wtf.expand_dims(inputs[:,12,:,:,2], -1),
                             labels[:,12], outputs[:,12]], 1)
        disp13 = wtf.concat([wtf.expand_dims(inputs[:,13,:,:,0], -1),
                             wtf.expand_dims(inputs[:,13,:,:,1], -1),
                             wtf.expand_dims(inputs[:,13,:,:,2], -1),
                             labels[:,13], outputs[:,13]], 1)
        disp14 = wtf.concat([wtf.expand_dims(inputs[:,14,:,:,0], -1),
                             wtf.expand_dims(inputs[:,14,:,:,1], -1),
                             wtf.expand_dims(inputs[:,14,:,:,2], -1),
                             labels[:,14], outputs[:,14]], 1)
        disp = wtf.concat([disp00, disp01, disp02, disp03, disp04, disp05, disp06,
                           disp07, disp08, disp09, disp10, disp11, disp12, disp13,
                           disp14], 2)
        
        wtf.summary.image('concat', disp, max_outputs=200)
    
        predictions = {'results': outputs}
    
        if mode == wtf.estimator.ModeKeys.PREDICT:
            export_outputs = {
                'predictions': wtf.estimator.export.PredictOutput(outputs)
            }
            return wtf.estimator.EstimatorSpec(
                mode=mode,
                predictions=outputs
            )
    
        # loss = wtf.losses.mean_squared_error(labels, outputs)
#         loss = wtf.reduce_sum(wtf.abs(labels - outputs), axis=[1,2,3])
        loss = wtf.reduce_sum(wtf.abs(labels - outputs), axis=[2,3,4])
        loss = wtf.reduce_mean(loss)
        
        global_step = wtf.train.get_global_step()
        lr = wtf.train.exponential_decay(params.learning_rate, global_step, 10000, 0.96, staircase=True)
        
        wtf.summary.scalar('learning_rate', lr)
#         wtf.summary.scalar('learning_rate', params.learning_rate)
        
        optimizer = wtf.train.AdamOptimizer(
#             learning_rate=params.learning_rate,
            learning_rate=lr,
            beta1=params.beta1,
            beta2=params.beta2
        )
    
        update_ops = wtf.get_collection(wtf.GraphKeys.UPDATE_OPS)
        with wtf.control_dependencies(update_ops):
            train_op = optimizer.minimize(
                loss=loss, global_step=wtf.train.get_global_step())
    
        eval_metric_ops = {
            "mse_i1": wtf.metrics.mean_squared_error(labels, inputs[:,:,:,:,0]),
            "mse_i2": wtf.metrics.mean_squared_error(labels, inputs[:,:,:,:,1]),
            "mse_i3": wtf.metrics.mean_squared_error(labels, inputs[:,:,:,:,2]),
            "mse_o": wtf.metrics.mean_squared_error(labels, outputs)
        }
        
        eval_summary_hook = wtf.train.SummarySaverHook(
                                save_steps=1,
                                output_dir= "summary/train/" + MODEL_NAME + "/eval",
                                scaffold=wtf.train.Scaffold(summary_op=wtf.summary.merge_all()))
        
        evaluation_hooks=[]
        evaluation_hooks.append(eval_summary_hook)
    
        estimator_spec = wtf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=eval_metric_ops,
            evaluation_hooks=evaluation_hooks
        )
        
        return estimator_spec

In [ ]:
def main(_):
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    
    wtf.logging.set_verbosity(wtf.logging.INFO)

    hparams = wtf.contrib.training.HParams(
        #num_epochs=1,
        #batch_size = 80,
        #forget_bias=1.0,
        learning_rate=0.0001,
        beta1=0.9,
        beta2=0.999
    )
    
    session_config = wtf.ConfigProto()
    session_config.gpu_options.allow_growth = True
    session_config.allow_soft_placement = True
    
    run_config = wtf.estimator.RunConfig(
        log_step_count_steps=10,
        save_summary_steps=10,
        tf_random_seed=19830610,
        model_dir=os.path.join('summary', 'train', MODEL_NAME),
        session_config=session_config
    )
    
    estimator = wtf.estimator.Estimator(
        model_fn=model_fn, params=hparams, config=run_config)
    
    train_dataset = du.create_dataset_tea5poon('train', train_data)
    train_dataset = train_dataset.shuffle(500)
    train_dataset = train_dataset.batch(BATCH_SIZE)
    train_dataset = train_dataset.repeat(EPOCHS)
    
    val_dataset = du.create_dataset_tea5poon('val', val_data)
    val_dataset = val_dataset.batch(BATCH_SIZE)
    val_dataset = val_dataset.repeat(1)
    
    test_dataset = du.create_dataset_tea5poon('test', test_data)
    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.repeat(1)
    
    def train_input_fn():
        train_iterator = train_dataset.make_one_shot_iterator()
        features, labels = train_iterator.get_next()
        return features, labels
    
    def val_input_fn():
        val_iterator = val_dataset.make_one_shot_iterator()
        features, labels = val_iterator.get_next()
        return features, labels
    
    def test_input_fn():
        test_iterator = test_dataset.make_one_shot_iterator()
        features, labels = test_iterator.get_next()
        return features, labels
    
    estimator.train(input_fn=train_input_fn, max_steps=None)
#     estimator.evaluate(input_fn=val_input_fn)
#     estimator.evaluate(input_fn=test_input_fn)

In [ ]:
if __name__ == '__main__':
    wtf.app.run()